### Install support packages

### Initialization

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, confusion_matrix

import cv2

import scikitplot as skplt

import os, sys, glob

import matplotlib.pyplot as plt

import numpy as np

%matplotlib inline

### Helper function

In [ ]:
def check_dir(path):
    if os.path.exists(path):
        print("Dir already exist")
    
    elif sys.platform == 'win32':
        os.system('mkdir ' + path)
    
    else:
        os.system('mkdir -p ' + path)
        print('New Path created : ', path)
    return 
    
# Define the routine the add a new last layer
def add_new_last_layer(base_model, nb_classes):
    print('Add last layer to the convnet..')
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(nb_classes, activation='relu')(x)  # new FC layer, random init
    predictions = Dense(nb_classes, activation='softmax')(x)  # new softmax layer
    model = Model(inputs=base_model.input, outputs=predictions)  # Combine the network
    return model

def get_im(path):
    # Load as grayscale
    img = cv2.imread(path, 0)
    # Reduce size
    resized = cv2.resize(img, (224, 224))
    return resized

def load_train():
    nb_classes = 2
    img_rows, img_cols = 224, 224
    X_train = []
    y_train = []

    path='/tf/data/data_OCT/dme/*.png'
    files = glob.glob(path)
    for fl in files:
        img = get_im(fl)
        X_train.append(img)
        y_train.append(1)

    path='/tf/data/data_OCT/normal/*.png'
    files = glob.glob(path)
    for fl in files:
        img = get_im(fl)
        X_train.append(img)
        y_train.append(0)

    X_train = np.array(X_train, dtype=np.uint8)
    y_train = np.array(y_train, dtype=np.uint8)
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    # train_data = train_data.transpose((0, 3, 1, 2))
    y_train = to_categorical(y_train, nb_classes)
    X_train = X_train.astype('float32')
    X_train /= 255
    print('Train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')

    return X_train, y_train

def split_validation_set(train, target, test_size):
    random_state = 51
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    print('Split train: ', len(X_train))
    print('Split valid: ', len(X_test))
    return X_train, X_test, y_train, y_test

### Load data

### Build model

### Train model

In [ ]:
# Prepare tensorboard
TB_LOG = './output/logs_tb'
check_dir(TB_LOG)

tensorboard = TensorBoard(log_dir=TB_LOG,
                          histogram_freq=0,
                          batch_size=32,
                          write_graph=True,
                          write_grads=False,
                          write_images=False,
                          embeddings_freq=0,
                          embeddings_layer_names=None,
                          embeddings_metadata=None)

In [ ]:
history = model.fit(X_train, Y_train, 
                    batch_size=32, 
                    epochs=5, 
                    verbose=1, 
                    validation_data=(X_test, Y_test))

# Save model
model.save('./output/my_model.h5') 
del model  # clear memory

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Load trained model
model = load_model('./output/my_model.h5')

In [ ]:
# Prediction
Y_pred=model.predict([X_test])

In [ ]:
# Performance report
target_names = ["normal", "dme"]
print(classification_report(np.argmax(Y_test, axis=1), np.argmax(Y_pred, axis=1), target_names=target_names))

In [ ]:
# Plot roc curve
skplt.metrics.plot_roc_curve(np.argmax(Y_test,axis=1), Y_pred)
plt.show()

In [ ]:
# Show confusion matrix
confusion_matrix(np.argmax(Y_test,axis=1), np.argmax(Y_pred,axis=1))
model.metric()

### Restart kernel